In [18]:
import openai
from openai import OpenAI

Getting the OpenAi API key from the environment variable `OPENAI_API_KEY`:

```python
import os
def get_openai_api_key():
    """Retrieve the OpenAI API key from environment variables."""
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OpenAI API key not found in environment variables.")
    return api_key
# Example usage
if __name__ == "__main__":
    try:
        api_key = get_openai_api_key()
        print("OpenAI API key retrieved successfully.")
    except ValueError as e:
        print(e)
```

### Modify the OpenAI API key in the environment variable `OPENAI_API_KEY`:
```python
import os
os.environ["OPENAI_API_KEY"] = "your-new-key"
```

In [26]:
import os
def get_openai_api_key():
    """Retrieve the OpenAI API key from environment variables."""
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OpenAI API key not found in environment variables.")
    return api_key
# Example usage
if __name__ == "__main__":
    try:
        api_key = get_openai_api_key()
        print("OpenAI API key retrieved successfully.")
    except ValueError as e:
        print(e)

OpenAI API key retrieved successfully.


In [27]:
client = OpenAI()

In [28]:
response = client.chat.completions.create(
    model='gpt-4o-mini', # using a more widely available model
    messages=[{'role':'user', 'content':'what is my first name'}]
)

In [29]:
# the reponse message we get from model by open ai
print(response.choices[0].message.content)

I'm sorry, but I don't have access to personal data about individuals unless it's shared in the conversation. If you'd like to share your first name, feel free!


--------------
Getting the data
--------------


In [ ]:
! pip install requests

In [ ]:
!pip install elasticsearch

## Q2. Indexing the data

In [33]:
from elasticsearch import Elasticsearch

In [34]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [30]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [36]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [37]:
from tqdm.auto import tqdm

In [38]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

In [43]:
query = 'How do execute a command on a Kubernetes pod?'

In [44]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                # "filter": {
                #     "term": {
                #         "course": "data-engineering-zoomcamp"
                #     }
                # }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [45]:
top_score = response['hits']['hits'][0]['_score']
print(f"The score for the top-ranking result is: {top_score}")

The score for the top-ranking result is: 44.50556


## Q4. Filtering


In [47]:
query = 'How do copy a file to a Docker container?'

In [48]:
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [49]:
response['hits']['hits'][2]

{'_index': 'course-questions',
 '_id': 'FrmYepcBnp-ygQrxnvAv',
 '_score': 59.812744,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

## Q5. Building a prompt

In [51]:
context_template = """
Q: {question}
A: {text}
""".strip()

records = response['hits']['hits']
contexts = []

for record in records:
    question = record['_source']['question']
    text = record['_source']['text']
    context = context_template.format(question=question, text=text)
    contexts.append(context)

context_combined = "\n\n".join(contexts)

question_query = "How do copy a file to a Docker container?"

# Prompt template
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
prompt = prompt_template.format(question=question_query, context=context_combined)
prompt_length = len(prompt)
print(f"The length of the resulting prompt is: {prompt_length}")

The length of the resulting prompt is: 1446


## Q6. Tokens

In [56]:
%pip install -q tiktoken
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

Note: you may need to restart the kernel to use updated packages.


In [57]:
tokens = encoding.encode(prompt)
num_tokens = len(tokens)
print(f"The number of tokens in the prompt is: {num_tokens}")

The number of tokens in the prompt is: 320
